In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import multiprocessing
from itertools import repeat


import matplotlib.pyplot as plt
%matplotlib inline


from pyomo.environ import *

In [2]:
N_n = 7
N_t = 96
feasibleTrip = np.load('Parameters/Chicago2017_RealCapacity/pubCharger/feasible_chain.npy')
N_e_total = feasibleTrip.shape[0]

q = sparse.load_npz('Parameters/Chicago2017_RealCapacity/pubCharger/q_rec.npz').toarray()[0][feasibleTrip]

E_init = 0.5
E_terminal = 0.5-1e-3
L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1
demand_cum = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/demand_cum.npz").toarray()[feasibleTrip]

N_e = 1000
groupNum = N_e_total // N_e
N_e_last = N_e_total - N_e * groupNum


Location = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/Location.npz")[feasibleTrip]


In [3]:
mapping = pd.read_csv('TripChains/county_Mapping.csv', index_col = 0)
mappings = {}
for i in range(len(mapping)):
    mappings[mapping['nodeIndex'][i]] = mapping['countyIndex'][i]
countyIndices = mapping['countyIndex'].unique()
countyNodes = {}
for n in range(len(countyIndices)):
    countyNodes[countyIndices[n]] = n

In [4]:
boolIndex = {}
for countyInd in countyIndices:
    boolIndex[countyInd] = np.zeros((N_e_total, N_t))
    for t in range(N_t):
        boolIndex[countyInd][:, t] = Location[:, t].toarray().T[0] == countyInd


def getCountyInd(x):
    if x == -1:
        return -1
    else:
        return countyNodes[x]

vectorized_f = np.vectorize(getCountyInd)
location_countyID = vectorized_f(Location.toarray())
    

In [6]:
'''problem for each group'''

model = ConcreteModel()

# Variables
model.x_e_UB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.x_e_LB = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.E_e_UB = Var(range(N_e), range(N_t), domain=NonNegativeReals, bounds=(0, 1))
model.E_e_LB = Var(range(N_e), range(N_t), domain=NonNegativeReals, bounds=(0, 1))
model.E_e_sum_weighted = Var(range(N_e), domain=NonNegativeReals, initialize=0)

model.x_capacity = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)

model.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model.power_UB_para = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.demand_cum = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)

# model.pene_para = Param(within=NonNegativeReals, mutable=True)
# model.omega_para = Param(range(N_e), within=NonNegativeReals, mutable=True)
# model.q = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# pene_para * q[e] * omega[e] 
model.coe_E = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model.coe_county0 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) 
model.coe_county1 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county2 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county3 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county4 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county5 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county6 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)



# Constraints for individual trip chain
model.c1 = ConstraintList()
for e in range(N_e):
    for t in range(N_t):
        model.c1.add(expr=model.x_e_UB[e, t] <= model.x_capacity[e, t])
        model.c1.add(expr=model.x_e_LB[e, t] <= model.x_capacity[e, t])
        model.c1.add(expr=model.x_capacity[e, t] <= model.power_UB_para[e, t])
        model.c1.add(expr=model.E_e_LB[e, t] <= model.E_e_UB[e, t])
        model.c1.add(expr=E_init - model.demand_cum[e, t] + 
                     sum(L[t, tt] * model.x_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_UB[e, t])
        model.c1.add(expr=E_init - model.demand_cum[e, t] + 
                     sum(L[t, tt] * model.x_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_LB[e, t])
    model.c1.add(expr=model.E_e_LB[e, N_t-1] >= E_terminal)
    model.c1.add(expr=model.E_e_sum_weighted[e] == model.coe_E[e] * sum(model.E_e_UB[e, t]-model.E_e_LB[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model.c2 = ConstraintList()
for t in range(N_t):
    model.c2.add(sum(model.coe_county0[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[0])
    model.c2.add(sum(model.coe_county1[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[1])
    model.c2.add(sum(model.coe_county2[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[2])
    model.c2.add(sum(model.coe_county3[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[3])
    model.c2.add(sum(model.coe_county4[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[4])
    model.c2.add(sum(model.coe_county5[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[5])
    model.c2.add(sum(model.coe_county6[e, t] * model.x_capacity[e, t] for e in range(N_e)) <= model.ell_capacity[6])

# Objective
model.obj = Objective(expr=sum(model.E_e_sum_weighted[e] for e in range(N_e)), sense=maximize)

In [7]:
'''problem for the last group'''

model_last = ConcreteModel()

# Variables
model_last.x_e_UB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.x_e_LB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.E_e_UB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, bounds=(0, 1))
model_last.E_e_LB = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, bounds=(0, 1))
model_last.E_e_sum_weighted = Var(range(N_e_last), domain=NonNegativeReals, initialize=0)

model_last.x_capacity = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)

model_last.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model_last.power_UB_para = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

model_last.demand_cum = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# model_last.q = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)
# model_last.pene_para = Param(within=NonNegativeReals, mutable=True)
# model_last.omega_para = Param(range(N_e_last), within=NonNegativeReals, mutable=True)

model_last.coe_E = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model_last.coe_county0 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) 
model_last.coe_county1 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county2 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county3 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county4 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county5 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county6 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# Constraints for individual trip chain
model_last.c1 = ConstraintList()
for e in range(N_e_last):
    for t in range(N_t):
        model_last.c1.add(expr=model_last.x_e_UB[e, t] <= model_last.x_capacity[e, t])
        model_last.c1.add(expr=model_last.x_e_LB[e, t] <= model_last.x_capacity[e, t])
        model_last.c1.add(expr=model_last.x_capacity[e, t] <= model_last.power_UB_para[e, t])
        model_last.c1.add(expr=model_last.E_e_LB[e, t] <= model_last.E_e_UB[e, t])
        model_last.c1.add(expr=E_init - model_last.demand_cum[e, t] + 
                     sum(L[t, tt] * model_last.x_e_UB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_UB[e, t])
        model_last.c1.add(expr=E_init - model_last.demand_cum[e, t] + 
                     sum(L[t, tt] * model_last.x_e_LB[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_LB[e, t])
    model_last.c1.add(expr=model_last.E_e_LB[e, N_t-1] >= E_terminal)
    model_last.c1.add(expr=model_last.E_e_sum_weighted[e] == 
                 model_last.coe_E[e] * sum(model_last.E_e_UB[e, t]-model_last.E_e_LB[e, t] for t in range(N_t)))

# Constraints for aggregate charging power capacity
model_last.c2 = ConstraintList()
    
for t in range(N_t):
    model_last.c2.add(expr=sum(model_last.coe_county0[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[0])
    model_last.c2.add(expr=sum(model_last.coe_county1[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[1])
    model_last.c2.add(expr=sum(model_last.coe_county2[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[2])
    model_last.c2.add(expr=sum(model_last.coe_county3[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[3])
    model_last.c2.add(expr=sum(model_last.coe_county4[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[4])
    model_last.c2.add(expr=sum(model_last.coe_county5[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[5])
    model_last.c2.add(expr=sum(model_last.coe_county6[e, t] * model_last.x_capacity[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[6])

# Objective
model_last.obj = Objective(expr=sum(model_last.E_e_sum_weighted[e] for e in range(N_e_last)), sense=maximize)

In [ ]:
scale = 6
capacity_county = pd.read_csv('geo/capacity_county_ordered.csv', index_col = 0)['EV_LOAD_CAPACITY_KW'].values/1000 / 56.5 * 1000
capacity_total = capacity_county.sum() * scale
peneNum = 10

proportionNum = 9
proportion_cook = np.linspace(0.1, 0.9, proportionNum)
share_restCounties = capacity_county[1:]/capacity_county[1:].sum()

capacity_distribution = np.zeros((proportionNum, N_n))
for p in range(proportionNum):
    capacity_distribution[p, 0] = capacity_total * proportion_cook[p]
    capacity_distribution[p, 1:] = capacity_total * share_restCounties * (1 - proportion_cook[p])




peneList = [0.5]
peneNum = 1

ratio_pubCharger_list = [0.4]
ratioNum = len(ratio_pubCharger_list)

In [ ]:
E_UB_rec = {}
E_LB_rec = {}
ell_UB_rec = {}

groupShare_matrix = sparse.load_npz(
                f"Parameters/Chicago2017_RealCapacity/pubCharger/groupShare/group.npz").toarray()

omega_matrix = sparse.load_npz(
    f"Results/Chicago2017_RealCapacity/omega_spatial/omega_pene=0.5_ratio=0.4.npz").toarray()

countyID_list = [31, 43, 89, 97, 111, 197, 93]

solver = SolverFactory('gurobi')
for i in range(peneNum):
    E_UB_rec[i] = {}
    E_LB_rec[i] = {}
    ell_UB_rec[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        E_UB_rec[i][ratio_pubCharger] = np.zeros((proportionNum, N_n*N_t))
        E_LB_rec[i][ratio_pubCharger] = np.zeros((proportionNum, N_n*N_t))
        ell_UB_rec[i][ratio_pubCharger] = np.zeros((proportionNum, N_n*N_t))
        p_max_total = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()[feasibleTrip]
        power_UB_para_value = p_max_total
    

        for p in range(proportionNum):
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, proportion_cook={proportion_cook[p]}")
            capacity_value = capacity_distribution[p]
            pene_value = peneList[i]
            omega_value = omega_matrix[p]
            groupShare = groupShare_matrix[i].reshape(groupNum+1, N_n)
            coe_temp = pene_value * q * omega_value
            model.pene_para = pene_value
            start_all = time.time()
            for g in range(groupNum):
                '''parameter seeting for each group'''
                for n in range(N_n):
                    model.ell_capacity[n] = capacity_value[n] * groupShare[g, n]

                for e in range(N_e):
                    model.coe_E[e] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e]
                    for t in range(N_t):
                        model.power_UB_para[e, t] = power_UB_para_value[g * N_e + e, t]
                        model.demand_cum[e, t] = demand_cum[g * N_e + e, t]
                        model.coe_county0[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[31][g * N_e + e, t]
                        model.coe_county1[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[43][g * N_e + e, t]
                        model.coe_county2[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[89][g * N_e + e, t]
                        model.coe_county3[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[97][g * N_e + e, t]
                        model.coe_county4[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[111][g * N_e + e, t]
                        model.coe_county5[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[197][g * N_e + e, t]
                        model.coe_county6[e, t] = pene_value * q[g * N_e + e] * omega_value[g * N_e + e] * boolIndex[93][g * N_e + e, t]

                '''solve the problem for each group'''
                start = time.time()
                results = solver.solve(model, warmstart=True)

                if str(results['Solver'][0]['Termination condition']) == 'optimal':
                    x_UB_result = np.array(model.x_e_UB[:, :]()).reshape(N_e, N_t)
                    x_LB_result= np.array(model.x_e_LB[:, :]()).reshape(N_e, N_t)
                    ell_UB_result = np.array(model.x_capacity[:, :]()).reshape(N_e, N_t)

                    E_UB_group = np.zeros((N_n, N_t))
                    E_LB_group = np.zeros((N_n, N_t))
                    ell_UB_group = np.zeros((N_n, N_t))
                    for e in range(N_e):
                        E_UB_county_temp = np.zeros((N_n, N_t))
                        E_LB_county_temp = np.zeros((N_n, N_t))
                        ell_UB_temp = np.zeros((N_n, N_t))
                        for n in range(N_n):
                            countyID = countyID_list[n]
                            E_UB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * x_UB_result[e] * 0.25
                            E_LB_county_temp[n] = boolIndex[countyID][g * N_e+e, :] * x_LB_result[e] * 0.25
                            ell_UB_temp[n] = boolIndex[countyID][g * N_e+e, :] * ell_UB_result[e]

                        E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                        E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                        E_UB_group += E_UB_county_temp * coe_temp[g * N_e + e]
                        E_LB_group += E_LB_county_temp * coe_temp[g * N_e + e]
                        ell_UB_group += ell_UB_temp * coe_temp[g * N_e + e]

                    E_UB_rec[i][ratio_pubCharger][p] += E_UB_group.reshape(-1) * 56.5 / 1000
                    E_LB_rec[i][ratio_pubCharger][p] += E_LB_group.reshape(-1) * 56.5 / 1000
                    ell_UB_rec[i][ratio_pubCharger][p] += ell_UB_group.reshape(-1) * 56.5 / 1000

                    gap_E = (E_UB_rec[i][ratio_pubCharger][p].reshape(N_n, N_t) - 
                             E_LB_rec[i][ratio_pubCharger][p].reshape(N_n, N_t)).sum(axis = 0)
                    print (f"group={g}, obj={gap_E[-1]}, time={time.time()-start}")


            '''solve the problem for the last group'''
            model_last.pene_para = pene_value
            for n in range(N_n):
                model_last.ell_capacity[n] = capacity_value[n] * groupShare[groupNum, n]

            for e in range(N_e_last):
                # model_last.omega_para[e] = omega_value[groupNum * N_e + e]
                # model_last.q[e] = q[groupNum * N_e + e]
                model_last.coe_E[e] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e]
                for t in range(N_t):
                    model_last.power_UB_para[e, t] = power_UB_para_value[groupNum * N_e + e, t]
                    model_last.demand_cum[e, t] = demand_cum[groupNum * N_e + e, t]
                    model_last.coe_county0[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[31][groupNum * N_e + e, t]
                    model_last.coe_county1[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[43][groupNum * N_e + e, t]
                    model_last.coe_county2[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[89][groupNum * N_e + e, t]
                    model_last.coe_county3[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[97][groupNum * N_e + e, t]
                    model_last.coe_county4[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[111][groupNum * N_e + e, t]
                    model_last.coe_county5[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[197][groupNum * N_e + e, t]
                    model_last.coe_county6[e, t] = pene_value * q[groupNum * N_e + e] * omega_value[groupNum * N_e + e] * boolIndex[93][groupNum * N_e + e, t]

            results_last = solver.solve(model_last)

            if str(results_last['Solver'][0]['Termination condition']) == 'optimal':
                x_UB_result = np.array(model_last.x_e_UB[:, :]()).reshape(N_e_last, N_t)
                x_LB_result= np.array(model_last.x_e_LB[:, :]()).reshape(N_e_last, N_t)
                ell_UB_result = np.array(model_last.x_capacity[:, :]()).reshape(N_e_last, N_t)

                E_UB_group = np.zeros((N_n, N_t))
                E_LB_group = np.zeros((N_n, N_t))
                ell_UB_group = np.zeros((N_n, N_t))
                for e in range(N_e_last):
                    E_UB_county_temp = np.zeros((N_n, N_t))
                    E_LB_county_temp = np.zeros((N_n, N_t))
                    ell_UB_temp = np.zeros((N_n, N_t))
                    for n in range(N_n):
                        countyID = countyID_list[n]
                        E_UB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * x_UB_result[e] * 0.25
                        E_LB_county_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * x_LB_result[e] * 0.25
                        ell_UB_temp[n] = boolIndex[countyID][groupNum * N_e+e, :] * ell_UB_result[e]

                    E_UB_county_temp = np.cumsum(E_UB_county_temp, axis = 1)
                    E_LB_county_temp = np.cumsum(E_LB_county_temp, axis = 1)

                    E_UB_group += E_UB_county_temp * coe_temp[groupNum * N_e + e]
                    E_LB_group += E_LB_county_temp * coe_temp[groupNum * N_e + e]
                    ell_UB_group += ell_UB_temp * coe_temp[groupNum * N_e + e]

                E_UB_rec[i][ratio_pubCharger][p] += E_UB_group.reshape(-1) * 56.5 / 1000
                E_LB_rec[i][ratio_pubCharger][p] += E_LB_group.reshape(-1) * 56.5 / 1000
                ell_UB_rec[i][ratio_pubCharger][p] += ell_UB_group.reshape(-1) * 56.5 / 1000

                gap_E_last = (E_UB_rec[i][ratio_pubCharger][p].reshape(N_n, N_t) - 
                              E_LB_rec[i][ratio_pubCharger][p].reshape(N_n, N_t)).sum(axis = 0)
                print(f"ratio={ratio_pubCharger}, proportion_cook={proportion_cook[p]}," \
                       f"pene={pene_value}, obj={(E_UB_rec[i][ratio_pubCharger][p] - E_LB_rec[i][ratio_pubCharger][p]).sum()}, time={time.time()-start_all}")
            else:
                print(f"ratio={ratio_pubCharger}, proportion_cook={proportion_cook[p]}, pene={pene_value}, infeasible")

            # break # only one proportion
        break # only one ratio
    break   # only one pene
                

        



start: EV_pene=0.5, ratio=0.4, proportion_cook=0.1
group=0, obj=1324.0899029026218, time=46.72569990158081
group=1, obj=2505.9427018279202, time=44.691136837005615
group=2, obj=3645.6581456201397, time=43.706218957901
group=3, obj=5126.193999814755, time=45.947402000427246
group=4, obj=6336.538307403452, time=48.940799951553345
group=5, obj=7207.218854396195, time=41.56040978431702
group=6, obj=8452.331626003837, time=43.69891405105591
group=7, obj=9646.882371604937, time=42.34162521362305
group=8, obj=10885.362420669919, time=42.80482602119446
group=9, obj=12175.83501142002, time=44.37206292152405
group=10, obj=13712.517147282533, time=41.88856291770935
group=11, obj=14307.606044810918, time=22.98607611656189
group=12, obj=14902.422518617599, time=21.339953899383545
group=13, obj=15400.46179188253, time=20.95497488975525
group=14, obj=16042.743666304274, time=21.02662181854248
group=15, obj=16600.615890788722, time=21.013993978500366
group=16, obj=17027.01600689874, time=20.9925270080

In [ ]:
import os
filePath = 'Results/Chicago2017_RealCapacity/AggregateModel_pubCharger'


for i in range(peneNum):
    pene_value = float(peneList[i])
    path_temp = filePath + f'/spatial_pene={pene_value:.1f}'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true 
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true
    if not isExists:
        os.makedirs(path_temp)  

    for ratio_pubCharger in ratio_pubCharger_list:
        for p in range(proportionNum):
            """save the energy bounds"""
            E_UB_sparse = sparse.csr_matrix(E_UB_rec[i][ratio_pubCharger][p])
            E_LB_sparse = sparse.csr_matrix(E_LB_rec[i][ratio_pubCharger][p])
            sparse.save_npz(f"{path_temp}/E_UB_ratio={ratio_pubCharger:.1f}_proportion_cook={proportion_cook[p]:.1f}.npz", E_UB_sparse)
            sparse.save_npz(f"{path_temp}/E_LB_ratio={ratio_pubCharger:.1f}_proportion_cook={proportion_cook[p]:.1f}.npz", E_LB_sparse)
            print (f"pene={peneList[i]:.1f}, ratio={ratio_pubCharger:.1f}, proportion_cook={proportion_cook[p]:.1f}, saved")
            # break # only one proportion
        break # only one ratio
    break # only one pene


pene=0.5, ratio=0.4, proportion_cook=0.1, saved
pene=0.5, ratio=0.4, proportion_cook=0.2, saved
pene=0.5, ratio=0.4, proportion_cook=0.3, saved
pene=0.5, ratio=0.4, proportion_cook=0.4, saved
pene=0.5, ratio=0.4, proportion_cook=0.5, saved
pene=0.5, ratio=0.4, proportion_cook=0.6, saved
pene=0.5, ratio=0.4, proportion_cook=0.7, saved
pene=0.5, ratio=0.4, proportion_cook=0.8, saved
pene=0.5, ratio=0.4, proportion_cook=0.9, saved


In [ ]:

capacity_county_MW = {}
for i in range(peneNum):
    pene_value = peneList[i]
    capacity_county_MW[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        capacity_county_MW[i][ratio_pubCharger] = {}
        p_max_total = sparse.load_npz(f"Parameters/Chicago2017_RealCapacity/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()[feasibleTrip]
        for p in range(proportionNum):
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, proportion_cook={proportion_cook[p]}")
            capacity_county_MW[i][ratio_pubCharger][p] = np.zeros((N_n, N_t))
            for n in range(N_n):
                for t in range(N_t):
                    county_key = countyIndices[n]
                    capacity_county_MW[i][ratio_pubCharger][p][n] = np.minimum((p_max_total * boolIndex[county_key]).T @ (q * pene_value) * 56.5/1000, capacity_distribution[p, n] * 56.5/1000)
            # break # only one proportion
        break # only one ratio
    break   # only one pene


start: EV_pene=0.5, ratio=0.4, proportion_cook=0.1
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.2
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.30000000000000004
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.4
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.5
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.6
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.7000000000000001
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.8
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.9


In [23]:
import os
filePath = 'Results/Chicago2017_RealCapacity/AggregateModel_pubCharger'


for i in range(4, peneNum):
    pene_value = float(peneList[i])
    path_temp = filePath + f'/spatial_pene={pene_value:.1f}'
    isExists=os.path.exists(path_temp) #判断路径是否存在，存在则返回true 
    if not isExists:
        os.makedirs(path_temp)  

    for ratio_pubCharger in ratio_pubCharger_list[3:]:
        for p in range(proportionNum):
            """save the updated power bound"""
            ell_UB_sparse = sparse.csr_matrix(capacity_county_MW[i][ratio_pubCharger][p].reshape(-1))
            sparse.save_npz(f"{path_temp}/ell_UB_new_ratio={ratio_pubCharger:.1f}_proportion_cook={proportion_cook[p]:.1f}.npz", ell_UB_sparse)
            print (f"start: EV_pene={peneList[i]}, ratio={ratio_pubCharger}, proportion_cook={proportion_cook[p]:.1f}")
            # break
        break
    break

start: EV_pene=0.5, ratio=0.4, proportion_cook=0.1
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.2
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.3
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.4
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.5
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.6
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.7
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.8
start: EV_pene=0.5, ratio=0.4, proportion_cook=0.9
